<h1> Lets Explore the neighborhoods in Toronto</h1>

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
df = pd.read_csv('Toronto_data_with_Geo.csv')

df.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [4]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [5]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [6]:
toronto_add='Toronto, Ontario' #CITY ADDRESS
geolocator=Nominatim(user_agent='toronto_explorer')
location=geolocator.geocode(toronto_add)
latitude=location.latitude
longitude=location.longitude
print("The coordinates of Toronto are {},{}".format(latitude,longitude))

The coordinates of Toronto are 43.6534817,-79.3839347


In [7]:
Toronto_map=folium.Map([latitude,longitude],zoom_start=10)
for lat,long,postalcode,borough,neighborhood in zip(df['Latitude'],df['Longitude'],df['PostalCode'],df['Borough'],df['Neighborhood']):
    label='Postal Code:{},Neighborhoods:{},Borough:{}'.format(postalcode,neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,long],radius=5,color='blue',popup=label,
                        fill=True,fill_color='#3186cc',fill_opacity=0.7,
                        parse_html=True).add_to(Toronto_map)

In [8]:
display(Toronto_map)

<h2>Only use the Borough which contains the word Toronto</h2>

In [9]:
df_tor = df[df['Borough'].str.contains('Toronto')]
df_tor.reset_index(inplace=True,drop=True)
df_tor.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [10]:
df_tor.shape

(39, 5)

In [11]:
df_tor['Borough'].value_counts()

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [20]:
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')   

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<h2>Lets prepare a new map with only the clusters for borough's with the name Toronto</h2>

In [24]:
#for set the cluster number as label number
kclusters=len(df_tor.Borough.unique())

# create map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough']):
    label = folium.Popup(str(df_tor['Borough']) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)
    
map_toronto

<h3>Now we have broken down just the clusters in Toronto with Toronto in the name. 
    We can see that North and East of downtown we have more spread out boroughs, 
    while in the downtown core we have much more conentrated boroughs</h3>